In [ ]:
import astropy.units as u
import numpy as np
import os
from astropy.table import Table
import copy
os.environ["PIXEDFIT_HOME"] = "/nvme/scratch/work/tharvey/piXedfit/"
from EXPANSE import ResolvedGalaxy, ResolvedGalaxies
from matplotlib import pyplot as plt
import matplotlib.patheffects as pe
import glob
from scipy import signal
from scipy.interpolate import interp1d
from scipy.stats import binned_statistic
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
# Change dpi to make plots larger

plt.rcParams["figure.dpi"] = 100

# Disable tex in matplotlib

plt.rcParams["text.usetex"] = False
%matplotlib inline

In [ ]:
import os
import shutil
from pathlib import Path

def reorganize_h5_files(input_folder, output_folder):
    """
    Copy and rename .h5 files from nested folders into a single folder.
    New names follow pattern: subfolder_name_original_name.h5
    
    Parameters
    ----------
    input_folder : str
        Path to input folder containing subfolders with .h5 files
    output_folder : str
        Path where renamed .h5 files will be copied
    """
    
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Convert to Path objects
    input_path = Path(input_folder)
    output_path = Path(output_folder)
    
    # Track number of files processed
    num_copied = 0
    
    # Iterate through all subfolders
    for subfolder in input_path.iterdir():
        if subfolder.is_dir():
            # Find all .h5 files in this subfolder
            h5_files = subfolder.glob('**/*.h5')
            
            for h5_file in h5_files:
                # Construct new filename
                new_name = f"{subfolder.name}_{h5_file.name}"
                new_path = output_path / new_name
                
                # Copy file
                shutil.copy2(h5_file, new_path)
                num_copied += 1
                print(f"Copied: {h5_file.name} -> {new_name}")
    
    print(f"\nFinished! Copied {num_copied} files to {output_folder}")

#reorganize_h5_files('/nvme/scratch/work/tharvey/EXPANSE/pipes/posterior/photoz_delayed/JOF_psfmatched', '/nvme/scratch/work/tharvey/EXPANSE/pipes/posterior/JOF_psfmatched/parallel_temp/')

In [ ]:
gal_id = '4040'
galaxy = ResolvedGalaxy.init_from_h5(f"JOF_psfmatched_{gal_id}.h5")

fig = galaxy.plot_bagpipes_overview(pipes_run_dir='/nvme/scratch/work/tharvey/EXPANSE/pipes/', photometry_to_show=["TOTAL_BIN", "1", "2", "3"], 
                              bagpipes_runs = {'CNST_SFH_RESOLVED':['RESOLVED', "1", "2", "3"]}, figsize=(12, 12), coloring_cmap='cmr.guppy')

fig.savefig(f'../plots/JOF_psfmatched_{gal_id}.pdf', dpi=300)

In [ ]:
galaxy = ResolvedGalaxy.init_from_h5("JOF_psfmatched_12443.h5")

galaxies = ResolvedGalaxies([galaxy])

#print(galaxy.sed_fitting_table['bagpipes']['CNST_SFH_RESOLVED_VORONOI'].meta)

In [ ]:
galaxy.calculate_mwa(resolved_name='BURSTY_SFH_RESOLVED_VORONOI')

galaxy.calculate_mwa(integrated_name='photoz_delayed')

In [ ]:
galaxies = ResolvedGalaxies(
    ResolvedGalaxy.init_all_field_from_h5("JOF_psfmatched", n_jobs = 6)
)

In [ ]:
# Select only galaxies within Ha range
redshifts = np.array([galaxy.redshift for galaxy in galaxies])
galaxies_ha = galaxies[redshifts < 6.6]
ew = {}
for galaxy in galaxies_ha:
    map, kwargs = galaxy.galfind_phot_property_map("EW", frame='rest', rest_optical_wavs=[4_200.0, 10_000.0] * u.AA, strong_line_names = 'Halpha', n_jobs=1, plot = False, iters=100, load_in=False)
    if map is not None:
        print(galaxy.photometry_properties['pixedfit']['EWrest_Halpha'])
        ew[galaxy.galaxy_id] = galaxy.photometry_properties['pixedfit']['EWrest_Halpha'][-6]
    

In [ ]:
run_name = 'CNST_SFH_RESOLVED'
region = 'pixedfit'
integrated_run = 'photoz_delayed'

delta_masses = {}
for galaxy in galaxies_ha:
    try:
        new_mass = galaxy.get_total_resolved_property(run_name, 'stellar_mass')[1]
    except Exception as e:
        print(e)
        continue
    integrated_mass = galaxy.sed_fitting_table['bagpipes'][integrated_run]['stellar_mass_50'][0]
    delta_mass = 10**new_mass / 10**integrated_mass
    delta_masses[galaxy.galaxy_id] = delta_mass

# Plot EW vs delta mass

for galaxy_id in ew:
    if galaxy_id not in delta_masses:
        continue
    ew_gal = np.median(ew[galaxy_id])

    plt.scatter(ew_gal, delta_masses[galaxy_id], color='black', s=5)

plt.xlabel('EW(Halpha) [A]')
plt.ylabel('Resolved mass / Integrated mass')






In [ ]:
match_dict = {'CNST_SFH_RESOLVED': 'pixedfit', 'CNST_SFH_RESOLVED_VORONOI': 'voronoi', 'BURSTY_SFH_RESOLVED_VORONOI': 'voronoi',
                'BURSTY_SFH_RESOLVED': 'pixedfit', 'CNST_SFH_RESOLVED_NOMIN': 'pixedfit_nomin', 'BURSTY_SFH_RESOLVED_NOMIN': 'pixedfit_nomin'}

for galaxy in galaxies:

    for key, binmap in match_dict.items():
        if key in galaxy.sed_fitting_table['bagpipes'].keys():
            meta = {'binmap_type': binmap}
            galaxy.sed_fitting_table['bagpipes'][key].meta = meta
    galaxy.dump_to_h5()

In [ ]:
binned_gals = galaxies.filter_single_bins('pixedfit')

all_binned = binned_gals.filter_single_bins('pixedfit_nomin')
all_binned = all_binned.filter_single_bins('voronoi')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
count = 0 

integrated_sfhs = ['photoz_delayed', 'photoz_lognorm', 'photoz_continuity', 'photoz_continuity_bursty', 'photoz_cnst']
colors = plt.cm.jet(np.linspace(0, 1, len(integrated_sfhs)))

resolved_name = 'CNST_SFH_RESOLVED_VORONOI'

for integrated_sfh, color in zip(integrated_sfhs, colors):
    start = 0
    for galaxy in binned_gals:

        if resolved_name not in galaxy.sed_fitting_table['bagpipes'].keys():
            continue
        
        try:
            x_percentiles = galaxy.calculate_mwa(resolved_name=resolved_name)
        except Exception as e:
            print(e)
            print("No resolved SFH", galaxy.galaxy_id)
            if galaxy.resolved_sfh is None:
                print("None")
                continue
            
            print(galaxy.resolved_sfh.keys())
            continue

        y_percentiles = galaxy.calculate_mwa(integrated_name=integrated_sfh)
        x = x_percentiles[1]
        y = y_percentiles[1]
        xerr_lower = x - x_percentiles[0]
        xerr_upper = x_percentiles[2] - x
        yerr_lower = y - y_percentiles[0]
        yerr_upper = y_percentiles[2] - y
        xerr = [[xerr_lower], [xerr_upper]]
        print(x_percentiles)
        yerr = [[yerr_lower], [yerr_upper]]
        xerr = np.abs(xerr)
        ax.plot(x, y, 'o', color=color, label=integrated_sfh if start == 0 else None)
        start += 1
        ax.errorbar(x, y, xerr=xerr, yerr=yerr, color=color, marker='none', linestyle='none', alpha=0.5, linewidth=0.5)
        #count += 1

ax.legend()
# color by mass discrepancy?
ax.set_yscale('log')
ax.set_xscale('log')
# Use normal locator not scientific notation
ax.xaxis.set_major_formatter(plt.ScalarFormatter())
ax.yaxis.set_major_formatter(plt.ScalarFormatter())

# Limits are approx 1 - 200, add a 1:1 line
ax.plot([3, 200], [3, 200], 'k--')

ax.set_xlabel("MWA (resolved)")
ax.set_ylabel("MWA (integrated)")

print(count)



In [ ]:
for galaxy in galaxies:
    if galaxy.resolved_mass is not None and 'photoz_delayed' in galaxy.resolved_mass.keys():
        galaxy.resolved_mass.pop('photoz_delayed')
        galaxy.dump_to_h5()

In [ ]:
run_name = 'CNST_SFH_RESOLVED'
for galaxy in galaxies:
    if run_name in galaxy.sed_fitting_table['bagpipes'].keys():
        galaxy.plot_bagpipes_sfh(
                            run_name,
                            bins_to_show=["RESOLVED"],
                            plot=False,
                            force=True,
                            run_dir='pipes/',
                        )

In [ ]:
run_name = 'CNST_SFH_RESOLVED'
region = 'pixedfit'

integrated_run = 'photoz_continuity'
x_param = 'ssfr_10myr_50'
x_vals = [galaxy.sed_fitting_table['bagpipes'][run_name][x_param][0] for galaxy in binned_gals]

mask = []
limit = 8.5
for galaxy in galaxies:
    try:
        new_mass = galaxy.get_total_resolved_property(run_name, 'stellar_mass')[1]
        mask.append(True if new_mass < limit else False)
    except:
        mask.append(False)

selected_galaxies = galaxies[mask]

delta_masses = []

for galaxy in binned_gals:
    try:
        new_mass = galaxy.get_total_resolved_property(run_name, 'stellar_mass')[1]
    except Exception as e:
        print(e)
        continue
    integrated_mass = galaxy.sed_fitting_table['bagpipes'][integrated_run]['stellar_mass_50'][0]
    delta_mass = 10**new_mass / 10**integrated_mass
    delta_masses.append(delta_mass)

delta_masses = np.array(delta_masses)

y_vals = delta_masses

print(y_vals)

assert len(x_vals) == len(y_vals), f"Lengths of x and y values do not match: {len(x_vals)} != {len(y_vals)}"
fig = binned_gals.plot_resolved_map_scatter(binmap_type='pixedfit', 
                                      run_name=run_name,
                                      scale_pixel_pkpc=20,#200, 
                                      cmap='inferno', 
                                      x_param = x_vals,
                                      y_param = y_vals,
                                      scale_physical=True,
                                      individual_cnorm=True,
                                      facecolor='black',
                                      norm_type='percentile',
                                      text_color='white', 
                                      scalebar_length=5,
                                      spread_factor=1,
                                      figsize=(25, 7));

ax = fig.get_axes()[0]
ax.set_xlim(-10, -6.5)


fig.savefig(f'../plots/{run_name}_resolved_map_scatter.pdf', dpi=300, bbox_inches='tight')


'''
binned_gals.plot_resolved_map_scatter(binmap_type='pixedfit', 
                                      run_name='CNST_SFH_RESOLVED', 
                                      map_param='sfr',
                                      scale_pixel_pkpc=20,#200, 
                                      cmap='RdYlBu',
                                      scale_physical=True,
                                      individual_cnorm=True,
                                      facecolor='black',
                                      norm_type='percentile',
                                      text_color='white', 
                                      scalebar_length=5,
                                      figsize=(25, 7));
'''

In [ ]:
galaxy = ResolvedGalaxy.init_from_h5("JOF_psfmatched_12443.h5")
galaxy.add_psf_models('/nvme/scratch/work/tharvey/EXPANSE/psfs/psf_models/star_stack/JOF_psfmatched/', file_ending='_psf.fits', overwrite=False)
galaxy.run_pysersic(show_plots=True, mask_type=None, make_diagnostic_plots=True, save_plots=True, fit_type='mvp', posterior_sample_method='mean', overwrite=True)


In [ ]:
index = galaxy.photometry_meta_properties['pixedfit']['beta_[1250,3000]AA']['PDF_keys'].index('TOTAL_BIN')

print(galaxy.photometry_properties['pixedfit']['beta_[1250,3000]AA'][index])

numbered_indexes = galaxy.photometry_meta_properties['pixedfit']['beta_[1250,3000]AA']['PDF_keys'][:index]
values = galaxy.photometry_properties['pixedfit']['beta_[1250,3000]AA'][:index]

print(np.min(values), np.max(values))


In [ ]:
values = [galaxy.redshift for galaxy in binned_gals]

values = [galaxy.sed_fitting_table['bagpipes']['photoz_delayed']['dust:Av_50'][0] for galaxy in binned_gals]
cmap = 'cmr.guppy'
label = rf'Dust Attenuation (mag)'  #rf'M$_{{\rm UV}}$'
from matplotlib.colors import LogNorm
norm = LogNorm(vmin=np.min(values), vmax=np.max(values))

print(np.min(values), np.max(values))

colors = plt.get_cmap(cmap)(norm(values))

fig, ax = plt.subplots(1, 1, figsize=(6, 6), facecolor='white', edgecolor='white')

for color, galaxy in zip(colors, binned_gals):
    index = galaxy.photometry_meta_properties['pixedfit']['beta_[1250,3000]AA']['PDF_keys'].index('TOTAL_BIN')

    beta_integrated = galaxy.photometry_properties['pixedfit']['beta_[1250,3000]AA'][index]

    numbered_indexes = galaxy.photometry_meta_properties['pixedfit']['beta_[1250,3000]AA']['PDF_keys'][:index]
    values = galaxy.photometry_properties['pixedfit']['beta_[1250,3000]AA'][:index]

    beta_max = np.max(values)
    beta_min = np.min(values)

    ax.plot([beta_integrated, beta_integrated], [beta_min, beta_max], alpha=0.7, color = color)
    

# draw a 1:1

xrange = ax.get_xlim()
yrange = ax.get_ylim()

minval = min(xrange[0], yrange[0])
maxval = max(xrange[1], yrange[1])

ax.set_xlim(xrange)
ax.set_ylim(yrange)


ax.plot([minval, maxval], [minval, maxval], 'k--', zorder=1)

ax.set_xlabel(rf'Integrated $\beta$')
ax.set_ylabel(rf'Resolved $\beta$')


dd = make_axes_locatable(ax)
cbar_ax = dd.append_axes('right', size='5%', pad=0.05)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax, label=label)
# Change colorbar to linear tick labels
cbar_ax.yaxis.set_major_formatter(plt.ScalarFormatter())

fig.savefig(f'../plots/beta_comparison.pdf', dpi=300, bbox_inches='tight')


In [ ]:
failed = 0
for galaxy in binned_gals:
    try:
        fig, ax = galaxy.galfind_phot_property_map(property='UV_Beta', rest_UV_wav_lims=[1_250., 3_000.] * u.AA, load_in=False, iters=1)
        fig.savefig(f'../galaxies/diagnostic_plots/beta/{galaxy.galaxy_id}_pixedfit_beta_map.png', bbox_inches='tight', dpi=200)
    except Exception as e:
        print(e)
        print(f"No beta map for {galaxy.galaxy_id}")
        failed += 1

print(f'Failed to make {failed} beta maps')

In [ ]:

fig, ax = None, None

for galaxy in binned_gals:
    for pos, (run, map, c) in enumerate(zip(['CNST_SFH_RESOLVED'], ['pixedfit'], ['r'])):
        fig,ax = galaxy.plot_resolved_scatter(run, binmap_type=map, x_param='stellar_mass', y_param='sfr_10myr', marker_color=c, log_x=False, log_y=True, fig=fig, axes=ax)

#ax.set_xlim(1e5, None)
#ax.set_ylim(1e-2, None)
#plot_resolved_scatter(galaxy, 'CNST_SFH_RESOLVED', log_y=True, x_param='stellar_mass_density', y_param='sfr_10myr_density', marker_color='r', log_x=True)

       

In [ ]:
# To Do - write total mass, SFR, ID redshift on plot. 
run = "CNST_SFH_RESOLVED_VORONOI"
binmap_type = "voronoi"

galaxy = ResolvedGalaxy.init_from_h5("JOF_psfmatched_12443.h5")

#fig = galaxy.plot_bagpipes_results(run_name=run, binmap_type=binmap_type, parameters=["lupton_rgb", "stellar_mass", "sfr", "mass_weighted_age", "dust:Av",  "beta_C94", "Halpha_EWrest"], area_norm=False, weight_mass_sfr=True,  rgb_q=1, rgb_stretch=7e-4, scale_border=0.1, max_on_row=8)
#fig.savefig(f'../galaxies/diagnostic_plots/bagpipes_results/{galaxy.galaxy_id}_{run}.pdf', dpi = 200, bbox_inches='tight')


for galaxy in galaxies:
    try:
        fig = galaxy.plot_bagpipes_results(run_name=run, binmap_type=binmap_type, parameters=["lupton_rgb", "stellar_mass", "sfr","mass_weighted_age", "dust:Av",  "beta_C94", "Halpha_EWrest"], area_norm=True, weight_mass_sfr=True,  rgb_q=1, rgb_stretch=3e-4, scale_border=0.2, max_on_row=8)
        fig.savefig(f'../galaxies/diagnostic_plots/bagpipes_results/{galaxy.galaxy_id}_{run}.png', dpi = 200, bbox_inches='tight')
        
        plt.show()
    except Exception as e:
        print(e)

#galaxy.measure_cog_of_property_map(run_name='CNST_SFH_RESOLVED_NOMIN', param='stellar_mass', binmap_type='pixedfit_nomin', plot=True, profile_type = 'differential_density')

In [ ]:
# Select only galaxies with resolved mass below 10*9 Msun
resolved_run = "CNST_SFH_RESOLVED"
binmap_type = 'voronoi'
galaxies = galaxies.filter_single_bins("pixedfit")
masses = [galaxy.get_total_resolved_property(resolved_run)[1] for galaxy in galaxies]
galaxies_lowmass = galaxies[np.array(masses) < 9]

print(f"Number of galaxies with resolved mass below 10^9 Msun: {len(galaxies_lowmass)}")

ngals = 0
nbins = 0
for galaxy in galaxies_lowmass:
    if 'CNST_SFH_RESOLVED_VORONOI' not in galaxy.sed_fitting_table['bagpipes'].keys():
        nbin = galaxy.get_number_of_bins(binmap_type)
        #print(galaxy.galaxy_id, nbin)
        ngals += 1
        nbins += nbin
        if binmap_type not in galaxy.photometry_table['star_stack'].keys():
            print(galaxy.galaxy_id)
            galaxy.measure_flux_in_bins(binmap_type = binmap_type)

print(f"Number of galaxies which haven't been fit: {ngals}")
print(f"Number of bins: {nbins}")

print(galaxies_lowmass[0].galaxy_id)

In [ ]:
single_galaxies = ResolvedGalaxies(ResolvedGalaxy.init_all_field_from_h5("JOF_psfmatched", n_jobs = 6, h5_folder='/nvme/scratch/work/tharvey/EXPANSE/galaxies/singlebin/'))

In [ ]:
for galaxy in single_galaxies:
    print(galaxy.galaxy_id)
    for map in ['pixedfit_nomin', 'voronoi']:
        try:
            galaxy.measure_flux_in_bins(binmap_type=map)
        except:
            print(f"Failed to measure flux in bins for {galaxy.galaxy_id} and {map}")

In [ ]:


'''
galaxy.add_det_galaxy_region(overwrite=False)

galaxy.pixedfit_processing(gal_region_use="detection")

galaxy.pixedfit_binning(animate=True, overwrite=True, 
                        animation_save_path= f'/nvme/scratch/work/tharvey/EXPANSE/galaxies/diagnostic_plots/{galaxy.galaxy_id}_pixedfit_binning.gif')


galaxy.voronoi_binning(SNR_reqs=7, galaxy_region='detection', overwrite=False,
                               use_only_widebands=False, plot=True, quiet=False,
                               ref_band = 'combined_average')

'''

In [ ]:
#galaxy.run_galfitm()
#galaxy.add_psf_models('/nvme/scratch/work/tharvey/EXPANSE/psfs/psf_models/star_stack/JOF_psfmatched/', file_ending='_psf.fits')
galaxy.run_autogalaxy(model_type='sersic', band='F444W', mask_type='circular', mask_radius=0.95)

In [ ]:
for galaxy in galaxies_not_outshined:
    if 'pixedfit_nomin' not in galaxy.photometry_table['star_stack'].keys():
        print(galaxy.galaxy_id)
        print(galaxy.photometry_table.keys())
        galaxy.measure_flux_in_bins(binmap_type='pixedfit_nomin')

In [ ]:
#table = galaxies.save_to_fits(overwrite=True, save=True)

table = Table.read('/nvme/scratch/work/tharvey/EXPANSE/galaxies.fits')


## Resolved Mass vs. Integrated Mass

In [ ]:

#rename 'CNST_SFH_RESOLVED_P' to 'CNST_SFH_RESOLVED'
import copy
for galaxy in galaxies:
    '''
    if 'CNST_RESOLVED_SFH' in galaxy.sed_fitting_table['bagpipes'].keys():
        print('fixed')
        galaxy.sed_fitting_table['bagpipes']['CNST_SFH_RESOLVED'] = copy.deepcopy(galaxy.sed_fitting_table['bagpipes']['CNST_RESOLVED_SFH'])
        galaxy.sed_fitting_table['bagpipes'].pop('CNST_RESOLVED_SFH')
        
    
    if (galaxy, 'resolved_mass') and getattr(galaxy, 'resolved_mass') is not None and 'CNST_SFH_RESOLVED_P' in galaxy.resolved_mass.keys():
        galaxy.resolved_mass['CNST_SFH_RESOLVED'] = copy.deepcopy(galaxy.resolved_mass['CNST_SFH_RESOLVED_P'])
        galaxy.resolved_mass.pop('CNST_SFH_RESOLVED_P')
        print('pop')

    if hasattr(galaxy, 'resolved_sfr_10myr') and getattr(galaxy, 'resolved_sfr_10myr') is not None and 'CNST_SFH_RESOLVED_P' in galaxy.resolved_sfr_10myr.keys():
        galaxy.resolved_sfr_10myr['CNST_SFH_RESOLVED'] = copy.deepcopy(galaxy.resolved_sfr_10myr['CNST_SFH_RESOLVED_P'])
        galaxy.resolved_sfr_10myr.pop('CNST_SFH_RESOLVED_P')
        print('pop')

    if hasattr(galaxy, 'resolved_sfr_100myr') and getattr(galaxy, 'resolved_sfr_100myr') is not None and 'CNST_SFH_RESOLVED_P' in galaxy.resolved_sfr_100myr.keys():
        galaxy.resolved_sfr_100myr['CNST_SFH_RESOLVED'] = copy.deepcopy(galaxy.resolved_sfr_100myr['CNST_SFH_RESOLVED_P'])
        galaxy.resolved_sfr_100myr.pop('CNST_SFH_RESOLVED_P')
        print('pop')

    if hasattr(galaxy, 'resolved_sed') and getattr(galaxy, 'resolved_sed') is not None and 'CNST_SFH_RESOLVED_P' in galaxy.resolved_sed.keys():
        galaxy.resolved_sed['CNST_SFH_RESOLVED'] = copy.deepcopy(galaxy.resolved_sed['CNST_SFH_RESOLVED_P'])
        galaxy.resolved_sed.pop('CNST_SFH_RESOLVED_P')
        print('pop')
    '''
    if hasattr(galaxy, 'resolved_sfh') and getattr(galaxy, 'resolved_sfh') is not None and 'CNST_SFH_RESOLVED_P' in galaxy.resolved_sfh.keys():
        galaxy.resolved_sfh['CNST_SFH_RESOLVED'] = copy.deepcopy(galaxy.resolved_sfh['CNST_SFH_RESOLVED_P'])
        galaxy.resolved_sfh.pop('CNST_SFH_RESOLVED_P')
        print('pop')

    galaxy.dump_to_h5()
       

In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_lognorm", "photoz_continuity", "photoz_cnst", "photoz_continuity_bursty", "photoz_delayed"]
labels = ['Lognormal', 'Delayed', 'Continuity', 'Constant', 'Continuity Bursty']
plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
norm = plt.Normalize( vmin = 4.5, vmax = 10.5)

for marker, bagpipes_run in zip(markers, bagpipes_runs):
    print(bagpipes_run)
    galaxies.comparison_plot(bagpipes_run, "CNST_SFH_RESOLVED", label = False, s = 4,
                            n_jobs = 1, cmap = 'viridis',
                            color_by='int_burstiness', ax = ax, marker = marker, fig = fig, add_colorbar = True if bagpipes_run == "photoz_lognorm" else False, norm = norm,
                            filter_single_bins_for_map = 'pixedfit')

# Plot 0.5 dex offset line
xlim = ax.get_xlim()
ylim = ax.get_ylim()

ax.plot(xlim, np.array(ylim)+0.5, color='black', linestyle='--', linewidth=1)
# Label the line with angle text near the top of the line
ax.text(9.35, 9.95, r'0.5 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

ax.plot(xlim, np.array(ylim)+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
ax.text(8.85, 9.95, r'1.0 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

# Fake a legend with the markers
points = []
for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
    point = ax.plot([], [], marker = marker, color = 'black', label = labels[pos])
    points.append(point[0])

ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Bagpipes Run')

ax.set_xlabel(r'Integrated Stellar Mass [$\log_{10}(M_{\odot})$]')
ax.set_ylabel(r'Resolved Stellar Mass [$\log_{10}(M_{\odot})$]')

fig.savefig('../plots/resolved_mass_comparison.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')

In [ ]:
for galaxy in galaxies:
    if 'sfr_10myr_50' not in galaxy.sed_fitting_table['bagpipes']['photoz_delayed'].colnames:
        galaxy.load_bagpipes_results('photoz_delayed', run_dir = '../pipes/')
        print(galaxy.galaxy_id)

In [ ]:
gal.load_bagpipes_results('photoz_delayed', run_dir='../pipes/')

In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_cnst", 'photoz_delayed', "photoz_lognorm", "photoz_continuity", "photoz_continuity_bursty"] # "photoz_delayed"
labels = ['Constant', 'Delayed', 'Lognormal', 'Continuity',  'Continuity Bursty'] # 'Delayed',
plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')
fig, axs = plt.subplots(1, 3, figsize=(12, 5), dpi = 200, tight_layout = True, facecolor = 'white', sharex = True, sharey = True)
norm = plt.Normalize( vmin = 0, vmax = 12)
# make axes locatable
cmap = 'cmr.gem'
color_by = 'int_burstiness'
ax = axs[-1]

cbar = make_axes_locatable(ax).append_axes(
                "right", size="5%", pad="2%"
            )

cbar = fig.colorbar(
    cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=cbar,
    orientation="vertical",
    label="Burstiness",
)


resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']
resolved_maps = ['pixedfit', 'pixedfit_nomin', 'voronoi']

for pos, (resolved_run, resolved_map, ax) in enumerate(zip(resolved_runs, resolved_maps, axs)):
    for marker, bagpipes_run in zip(markers, bagpipes_runs):
        galaxies.comparison_plot(bagpipes_run, resolved_run, label = False, s = 15,
                                edgecolor = 'black', linewidths = 0.5, n_jobs = 1, cmap = cmap,
                                color_by=color_by, ax = ax, marker = marker, fig = fig, add_colorbar = False, norm = norm,
                                filter_single_bins_for_map = resolved_map, skip_missing = True)

    # Plot 0.5 dex offset line
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.set_xlabel('')
    ax.set_ylabel('')

    ax.plot(xlim, np.array(ylim)+0.5, color='black', linestyle='--', linewidth=1)
    # Label the line with angle text near the top of the line
    ax.text(9.35, 9.95, r'0.5 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

    ax.plot(xlim, np.array(ylim)+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
    ax.text(8.85, 9.95, r'1.0 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')
    
    if pos > 0:
        # Shade region above 9 on x-axis lightly
        ax.fill_between([9, 10], 0, 10, color = 'red', alpha = 0.1, ec='face', linewidth=0.00001)


    if pos == 0:
        # Fake a legend with the markers
        points = []
        for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
            point = ax.plot([], [], marker = marker, color = 'black', label = labels[pos])
            points.append(point[0])

        ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Integrated SFH')

    ax.text(0.98, 0.07, f"{resolved_map.replace('_', ' ').title()} Binning", transform=ax.transAxes, fontsize=15, ha='right', va='top', path_effects=[pe.withStroke(linewidth=2, foreground='white')], color='black')

fig.supxlabel(r'Integrated Stellar Mass [$\log_{10}(M_{\odot})$]', fontsize = 15, ha = 'center', va = 'center', y=0.05)
fig.supylabel(r'Resolved Stellar Mass [$\log_{10}(M_{\odot})$]', fontsize = 15)


fig.savefig('../plots/resolved_mass_comparison_combined.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')

## Mass Discrepancy as a Function of sSFR

In [ ]:
# use the table of results this time - loop over each SFH history again, plot sSFR vs delta mass. Propogate errors through the calculation.
#plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')

# SHIFT TO USING RESOLVED SSFRS!
import statsmodels.api as sm
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_cnst", "photoz_lognorm", "photoz_delayed", "photoz_continuity", "photoz_continuity_bursty"]
labels = ['Constant','Lognormal', 'Delayed', 'Continuity',  'Continuity Bursty']
plt.style.use('/nvme/scratch/work/tharvey/scripts/paper.mplstyle')
plot_dex_diff = True

delta_masses = {}
integrated = {}

colors = [
    "#D14D41",
    "#DA702C",
    #"#D0A215",
    "#879A39",
    "#3AA99F",
    "#4385BE",
    "#8B7EC8",
    "#CE5D97",
]

# Get colors from a colormap insead

cmap = 'cmr.tropical'
ncolors = len(bagpipes_runs)
colors = [plt.get_cmap(cmap)(1. * i / ncolors) for i in range(ncolors)]

resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']
rlabels = ['Constant SFH, Pixedfit Binning', 'Constant SFH, Pixedfit Nomin Binning', 'Constant SFH, Voronoi Binning']
fig, axs = plt.subplots(len(resolved_runs), 1, figsize=(7, 3*len(resolved_runs)),
dpi = 200, tight_layout = True, facecolor = 'white', sharex=True)

for rpos, (resolved_run, ax, rlabel) in enumerate(zip(resolved_runs, axs, rlabels)):
    for marker, bagpipes_run, color in zip(markers, bagpipes_runs, colors):
        if bagpipes_run == 'photoz_dpl':
            # Something weird going on with this run, skip it for now
            continue
        delta_mass = table[f'{resolved_run}_resolved_mass'][:, 1] - table[f'{bagpipes_run}_stellar_mass_50']

        delta_masses[bagpipes_run] = delta_mass
        integrated[bagpipes_run] = table[f'{bagpipes_run}_stellar_mass_50']
        

        upper_error_resolved = table[f'{resolved_run}_resolved_mass'][:, 2] - table[f'{resolved_run}_resolved_mass'][:, 1]
        lower_error_resolved = table[f'{resolved_run}_resolved_mass'][:, 1] - table[f'{resolved_run}_resolved_mass'][:, 0]

        upper_error_bagpipes = table[f'{bagpipes_run}_stellar_mass_84'] - table[f'{bagpipes_run}_stellar_mass_50']
        lower_error_bagpipes = table[f'{bagpipes_run}_stellar_mass_50'] - table[f'{bagpipes_run}_stellar_mass_16']

        # Average the errors
        error_resolved = (upper_error_resolved + lower_error_resolved) / 2
        error_bagpipes = (upper_error_bagpipes + lower_error_bagpipes) / 2

        # Propogate errors
        error = np.sqrt(error_resolved**2 + error_bagpipes**2)


        if not plot_dex_diff:
            # Instead plot ratio of resolved mass to bagpipes mass
            delta_mass =  10**table[f'{bagpipes_run}_stellar_mass_50']/10**table['_resolved_mass'][:, 1] 
            error = np.abs(delta_mass) * np.sqrt((error / table[f'{bagpipes_run}_stellar_mass_50'])**2 + (error / table[f'{resolved_run}_resolved_mass'][:, 1])**2)

        yerr = error #placeholder for now

        xerr_lower = table[f'{bagpipes_run}_ssfr_10myr_50'] - table[f'{bagpipes_run}_ssfr_10myr_16']
        xerr_upper = table[f'{bagpipes_run}_ssfr_10myr_84'] - table[f'{bagpipes_run}_ssfr_10myr_50']

        xerr = np.array([xerr_lower, xerr_upper])
        xerr[xerr < 0] = 0

        ax.errorbar(table[f'{bagpipes_run}_ssfr_10myr_50'], delta_mass, yerr = yerr, xerr=xerr, marker='none', linestyle='none', alpha = 0.3, zorder=5, color = color, linewidth = 1)
        ax.scatter(table[f'{bagpipes_run}_ssfr_10myr_50'], delta_mass, marker=marker, s=15, edgecolors='black', linewidth=0.5, zorder=10, color = color)

        # Add a LOWESS line
        x = table[f'{bagpipes_run}_ssfr_10myr_50']
        y = delta_mass
    

    # Fix the x-axis limits
    ax.set_xlim(ax.get_xlim())
    ax.set_ylim(ax.get_ylim())

    # Make a dummy legend
    points = []
    for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
        point = ax.plot([], [], marker = marker, mec = 'black', label = labels[pos], color=colors[pos])
        points.append(point[0])

    ax.text(0.01, 0.96, rlabel, transform=ax.transAxes, fontsize=13, ha='left', va='top', path_effects=[pe.withStroke(linewidth=2, foreground='white')], color='black')
    if rpos == len(resolved_runs) - 1:
        ax.set_xlabel(r'Integrated Specific Star Formation Rate [10 Myr, $\log_{10}(\mathrm{yr}^{-1})$]', fontsize=11)
    if plot_dex_diff:
        ax.set_ylabel(r'Stellar Mass Offset [$\log_{10}(M_{\odot})$]', fontsize=11)
    else:
        ax.set_ylabel(r'Integrated / Resolved Stellar Mass')
    # Plot 1:1, 0.5 and 1 dex lines
    xlim = ax.get_xlim()


    ax.plot(xlim, [0, 0], color='black', linestyle='--', linewidth=1)

    if plot_dex_diff:
        ax.plot(xlim, np.array([0, 0])+0.5, color='black', linestyle='--', linewidth=1)
        #ax.text(-9.9, 0.62, r'0.5 dex', rotation=0, fontsize=8, color='black', ha = 'left', va = 'top')

        ax.plot(xlim, np.array([0, 0])+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
        #ax.text(-9.9, 1.12, r'1.0 dex', rotation=0, fontsize=8, color='black', ha = 'left', va = 'top')
    else:
        # Plot 1:1 line
        ax.plot(xlim, [1, 1], color='black', linestyle='--', linewidth=1)
        #ax.text(-9.9, 1, r'1:1', rotation=0, fontsize=8, color='black', ha = 'left', va = 'bottom')


    if not plot_dex_diff:
        # log scale for y-axis
        ax.set_ylim(0.001, 2)

        #ax.set_yscale('log')
    if rpos == len(resolved_runs) - 1:
        ax.legend(handles = points, loc = 'lower left', fontsize = 8, title = 'Integrated Bagpipes Run', frameon = True, title_fontsize = 8, markerscale = 0.7, labelspacing=0.1, fancybox = False, edgecolor = 'black')
    ax.set_xlim(-10, -6.9)
    #ax.set_ylim(-0.6, 1.5)

    #ax.set_title(resolved_run)

if plot_dex_diff:
    fig.savefig('../plots/ssfr_vs_mass_offset.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')



In [ ]:
for key in delta_masses.keys():
    print(key)
    print(np.mean(delta_masses[key]))
    print(np.std(delta_masses[key]))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
# Plot number of bins vs mass offset
regions = ['pixedfit', 'pixedfit_nomin', 'voronoi']
resolved_run = 'CNST_SFH_RESOLVED'
for galaxy in galaxies:
    bins = []
    for region in regions:
        nbins = galaxy.get_number_of_bins(region)
        bins.append(nbins)
    
    delta_masses = []
    for bagpipes_run in bagpipes_runs:
        print(bagpipes_run)
        new_mass = galaxy.get_total_resolved_property(resolved_run, 'stellar_mass')
        integrated_mass = galaxy.sed_fitting_table['bagpipes'][bagpipes_run]['stellar_mass_50'][0]
        delta_mass = new_mass[1] - integrated_mass
        delta_masses.append(delta_mass)

    print(bins, delta_masses)
    ax.plot(bins, delta_masses, 'o', color='black', markersize=5)
        


In [ ]:
# select photoz_lognorm which have delta_masses > 0.2 dex

select_pos = np.zeros(len(galaxies), dtype=bool)
select_pos2 = np.zeros(len(galaxies), dtype=bool)


# Select outshined galaxies
bagpipes_runs = ["photoz_lognorm", "photoz_delayed", "photoz_continuity"]

for key in bagpipes_runs:
    select_pos = select_pos | (delta_masses[key] > 0.5)
    select_pos2 = select_pos2 | ((integrated[key] < 8) & (delta_masses[key] < 0.2))

print(np.sum(select_pos))
print(np.sum(select_pos2))

select_pos2 = select_pos2 & ~select_pos
# remove duplicates from select_pos2


galaxy_ids = table['galaxy_id'][select_pos]
galaxy_ids_notoutshined = table['galaxy_id'][select_pos2]


galaxies_outshined = ResolvedGalaxies([galaxy for galaxy in galaxies if galaxy.galaxy_id in galaxy_ids])

galaxies_not_outshined = ResolvedGalaxies([galaxy for galaxy in galaxies if galaxy.galaxy_id in galaxy_ids_notoutshined])

#galaxies_notoutshined = ##
other_galaxies = ResolvedGalaxies([galaxy for galaxy in galaxies if galaxy.galaxy_id not in galaxy_ids_notoutshined and galaxy.galaxy_id not in galaxy_ids])


In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_lognorm", "photoz_delayed", "photoz_continuity", "photoz_cnst", "photoz_continuity_bursty"]
labels = ['Lognormal', 'Delayed', 'Continuity', 'Constant', 'Continuity Bursty']

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
norm = plt.Normalize( vmin = 0.1, vmax = 12)

for marker, bagpipes_run in zip(markers, bagpipes_runs):
    galaxies_not_outshined.comparison_plot(bagpipes_run, "BURSTY_SFH_RESOLVED_NOMIN", label = False, markersize = 4,
                            markeredgecolor = 'black', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'cmr.guppy',
                            color_by='int_burstiness', ax = ax, marker = marker, fig = fig, add_colorbar = True if bagpipes_run == "photoz_lognorm" else False, norm = norm) 
    try:
        galaxies_outshined.comparison_plot(bagpipes_run, "BURSTY_SFH_RESOLVED_NOMIN", label = False, markersize = 4,
                                markeredgecolor = 'deeppink', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'cmr.guppy',
                                color_by='int_burstiness', ax = ax, marker = marker, fig = fig, add_colorbar = False, norm = norm) 
    except KeyError:
        pass

# Plot 0.5 dex offset line
xlim = ax.get_xlim()
ylim = ax.get_ylim()

cax = fig.get_axes()[1]
cax.set_xlabel('Burstiness')

ax.plot(xlim, np.array(ylim)+0.5, color='black', linestyle='--', linewidth=1)
# Label the line with angle text near the top of the line
ax.text(9.35, 9.95, r'0.5 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

ax.plot(xlim, np.array(ylim)+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
ax.text(8.85, 9.95, r'1.0 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

# Fake a legend with the markers
points = []
for pos, (marker, bagpipes_run) in enumerate(zip(markers, bagpipes_runs)):
    point = ax.plot([], [], marker = marker, color = 'black', label = labels[pos])
    points.append(point[0])

leg = ax.legend(handles = points, loc = 'lower right', fontsize = 8, title = '   Integrated\nBagpipes Run')

# another fake legend to show outline color shows outshined vs not outshined

points = []
point = ax.plot([], [], marker = 'o', color = 'white', label = 'Outshined', markeredgecolor = 'deeppink', markeredgewidth = 0.5)
points.append(point[0])
point = ax.plot([], [], marker = 'o', color = 'white', label = 'Not Outshined', markeredgecolor = 'black', markeredgewidth = 0.5)
points.append(point[0])

ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Galaxy Type')

ax.add_artist(leg)


ax.set_xlabel(r'Integrated Stellar Mass [$\log_{10}(M_{\odot})$]', fontsize=11)
ax.set_ylabel(r'Resolved Stellar Mass (Voronoi Binning) [$\log_{10}(M_{\odot})$]', fontsize=11)


fig.savefig('../plots/resolved_mass_comparison_voronoi_bursty.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')

In [ ]:
map = 'voronoi'
nbins = 0
non_1 = 0
print(len(other_galaxies))
for galaxy in other_galaxies:
    try:
        bins = galaxy.get_number_of_bins(binmap_type = map)
        if bins > 1:
            non_1 += 1
    except TypeError:
        continue
   

    nbins += bins
    
print(nbins, non_1)


In [ ]:
failed = 0 
for galaxy in single_galaxies:
    #galaxy.add_det_galaxy_region(overwrite=True)
    try:
        if 'voronoi' not in galaxy.photometry_table['star_stack'].keys():
            
           
            print(f'Rerunning galaxy {galaxy.galaxy_id}')
            galaxy.voronoi_binning(SNR_reqs=7, galaxy_region='detection', overwrite=True,
                                use_only_widebands=False, plot=True, quiet=False,
                                ref_band = 'combined_average')
        
            galaxy.measure_flux_in_bins(binmap_type='voronoi')

    except (IndexError, ValueError) as e:
        print(e)
        import traceback
        traceback.print_exc()
        print(f"Failed for galaxy {galaxy.galaxy_id}")
        failed += 1

print(f'Failed for {failed} galaxies')

In [ ]:
nrows = len(galaxies_outshined)
ncols = 4

print(nrows)
positions_to_plot = [1, 4, 5, 9, 10, 19, 26, 27]
nrows = len(positions_to_plot)

# Calculate optimal figure size given the number of rows and columns, and the fact that all axis are square

figsize = (ncols * 1.5, nrows * 1.5)

# import make_axes_locatable
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, dpi = 200, tight_layout = True, facecolor = 'white', sharex=False, sharey=False)
# disable axis tick labels

print(np.shape(axes))
for ax in axes.flatten():
    ax.axis('off')

# Label columns
for i, label in enumerate(['RGB', 'piXedfit', 'piXedfit (no min)', 'Voronoi']):
    axes[0, i].text(0.5, 1.05, label, transform=axes[0, i].transAxes, ha='center', va='bottom', fontsize=10)
     

for i, galaxy in enumerate(galaxies_outshined):

    if i not in positions_to_plot:
        continue
    else:
        i = positions_to_plot.index(i)

    ax = axes[i, 0]
    rgb_stretch = 5e-4
    rgb_q = 5
    galaxy.plot_lupton_rgb(ax=ax, fig=fig,show=False, 
            red=["F444W"],
            green=["F356W"],
            blue=["F200W"],
            return_array=False,
            stretch=rgb_stretch,
            q=rgb_q,
            label_bands=True,
            add_compass=True if i == 0 else False,
            add_scalebar=True,
            compass_center = (17, 25),
            text_fontsize='x-small',
        )


    ax = axes[i, 1]
    #cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
    


    map = galaxy.pixedfit_map
    print(galaxy.meta_properties["zbest_fsps_larson_zfree"], galaxy.meta_properties["zbest_16_fsps_larson_zfree"], galaxy.meta_properties["zbest_84_fsps_larson_zfree"])
    mappable = ax.imshow(map, origin='lower', cmap='nipy_spectral_r', vmin=np.min(map), vmax=np.max(map))
    ax.text(0.05, 0.05, f'{galaxy.galaxy_id}\n z={galaxy.redshift:.2f}$^{{+{galaxy.meta_properties["zbest_84_fsps_larson_zfree"][0]-galaxy.meta_properties["zbest_fsps_larson_zfree"]:.2f}}}_{{-{galaxy.meta_properties["zbest_fsps_larson_zfree"]-galaxy.meta_properties["zbest_16_fsps_larson_zfree"][0]:.2f}}}$', transform=ax.transAxes, ha='left', va='bottom', fontsize=8,
              path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.pixedfit_map))-1}  ', 
            transform=ax.transAxes, ha='left', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    #cbar = fig.colorbar(mappable, cax=cbar_ax)

    ax = axes[i, 2]
    #cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
    minmap = galaxy.pixedfit_nomin_map
    mappable = ax.imshow(minmap, origin='lower', cmap='nipy_spectral_r', vmin=np.min(minmap), vmax=np.max(minmap))
    # Label with ID and number of bins
    ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.pixedfit_nomin_map))}', transform=ax.transAxes, ha='left', va='top', fontsize=8,
            path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    #cbar = fig.colorbar(mappable, cax=cbar_ax)

    ax = axes[i, 3]

    try:
        #cbar_ax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
        voronoi_map = galaxy.voronoi_map
        mappable = ax.imshow(voronoi_map, origin='lower', cmap='nipy_spectral_r', vmin=np.min(voronoi_map), vmax=np.max(voronoi_map))
        # Label with ID and number of bins
        ax.text(0.05, 0.95, f'N$_{{\\rm{{bins}}}}$: {len(np.unique(galaxy.voronoi_map))}', transform=ax.transAxes, ha='left', va='top', fontsize=8,
                path_effects=[pe.withStroke(linewidth=1, foreground='white')])
        #cbar = fig.colorbar(mappable, cax=cbar_ax)
    except AttributeError:
        # remove the axis
        ax.axis('off')

plt.subplots_adjust(wspace=-0.2)

plt.show()  


In [ ]:
# mass comparison for galaxies_outshined

fig, ax = plt.subplots(1, 1, figsize=(7, 7), dpi = 200, tight_layout = True, facecolor = 'white')
ax.set_title('Outshining')
galaxies_outshined.comparison_plot('CNST_SFH_RESOLVED_P', "CNST_SFH_RESOLVED_NOMIN", label = False, markersize = 4, 
                            markeredgecolor = 'black', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'viridis',
                            color_by='redshift', ax = ax, marker = 's', fig = fig, add_colorbar = True, norm = norm) 

fig, ax = plt.subplots(1, 1, figsize=(7, 7), dpi = 200, tight_layout = True, facecolor = 'white')
ax.set_title('No Outshining')
galaxies_not_outshined.comparison_plot('CNST_SFH_RESOLVED_P', "CNST_SFH_RESOLVED_NOMIN", label = False, markersize = 4, 
                            markeredgecolor = 'black', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, cmap = 'viridis',
                            color_by='redshift', ax = ax, marker = 'o', fig = fig, add_colorbar = True, norm = norm) 




In [ ]:
for galaxy in galaxies_not_outshined:
    print(galaxy.galaxy_id)
    print(galaxy.sed_fitting_table['bagpipes']['CNST_SFH_RESOLVED_NOMIN'])


In [ ]:
test_galaxies = galaxies_outshined

In [ ]:

fig = plt.figure(figsize=(4, 10), dpi = 200, tight_layout = True, facecolor = 'white')

axes = fig.subplots(len(test_galaxies), 1, sharex=True)

resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']
# nice color palette
resolved_colors = ['tab:blue', 'tab:orange', 'tab:green']

integrated_runs = ['photoz_lognorm', 'photoz_delayed', 'photoz_continuity', 'photoz_cnst', 'photoz_continuity_bursty']
# uniuqe cmap - not viridis
integrated_cmap = 'inferno'
# get colors from cmap
cmap = plt.get_cmap(integrated_cmap)
integrated_colors = [cmap(i) for i in np.linspace(0, 1, len(integrated_runs))]
resolved_run = "CNST_SFH_RESOLVED_VORONOI"

for i, galaxy in enumerate(test_galaxies):
    ax = axes[i]

    for resolved_run, resolved_color in zip(resolved_runs, resolved_colors):
        _, _ = galaxy.plot_bagpipes_sfh(run_name = resolved_run, axes = ax, bins_to_show=['RESOLVED'], fig = fig, resolved_color=resolved_color, time_unit='yr')
    
    for integrated_run, integrated_color in zip(integrated_runs, integrated_colors):
        _, _ = galaxy.plot_bagpipes_sfh(run_name = integrated_run, axes = ax, bins_to_show=['TOTAL_BIN'], fig = fig, marker_colors = [integrated_color], time_unit='yr')
    
    # Delete legend
    ax.get_legend().remove()

    ax.set_xscale('log')
    ax.set_xlabel('')
    # Label with galaxy ID and redshift

    ax.text(0.05, 0.95, f'{galaxy.galaxy_id}\nz={galaxy.redshift:.2f}', transform=ax.transAxes, ha='left', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    
    # Write resolved stellar mass 

    val = galaxy.resolved_mass[resolved_run][1]
    upper = galaxy.resolved_mass[resolved_run][2] - val
    lower = val - galaxy.resolved_mass[resolved_run][0]

    ax.text(0.97, 0.95, f'$\log_{{10}}(M_{{\star, \mathrm{{resolved}}}}) = {val:.2f}^{{+{upper:.2f}}}_{{-{lower:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')])

    ax.set_xlim(1e6, 5e8)

# Add a dummy legend with SFHs

points = []
for pos, (resolved_run, resolved_color) in enumerate(zip(resolved_runs, resolved_colors)):
    point = ax.plot([], [], color = resolved_color, label = resolved_run)
    points.append(point[0])

for pos, (integrated_run, integrated_color) in enumerate(zip(integrated_runs, integrated_colors)):
    point = ax.plot([], [], color = integrated_color, label = integrated_run)
    points.append(point[0])

fig.legend(handles = points, loc = 'lower center', fontsize = 8, title = 'SFH Type', title_fontsize = 8, markerscale = 0.7, labelspacing=0.1, ncol=2, bbox_to_anchor=(0.5, -0.07))

fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/sfh_comparison_outshined.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')


In [ ]:
run_dir = '/nvme/scratch/work/tharvey/EXPANSE/pipes/'
for galaxy in galaxies_outshined:
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), dpi = 200, tight_layout = True, facecolor = 'white')
    fig, _ = galaxy.plot_bagpipes_sfh(run_name = 'CNST_SFH_RESOLVED', bins_to_show=['RESOLVED'], run_dir=run_dir, fig=fig, axes=ax)
    ax = fig.get_axes()[0]
    
    fig, _ = galaxy.plot_bagpipes_sfh(run_name = 'photoz_delayed', bins_to_show=['TOTAL_BIN'], run_dir=run_dir, fig=fig, axes=ax)
    #print(ax)
    ax.set_xlim(0, 0.1)
    ax.set_xlabel('Lookback Time [Gyr]')
    ax.set_ylabel('SFR [M$_{\odot}$ yr$^{-1}$]')
    
    plt.show()

In [ ]:
for galaxy in other_galaxies:
    galaxy.pixedfit_processing(gal_region_use="detection", overwrite=True)
    galaxy.pixedfit_binning(save_out = True, SNR_reqs=4, Dmin_bin=1, redc_chi2_limit=100.0, del_r=1.0, overwrite=False, name_out="pixedfit_nomin")
    

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
tot_bins = 0
for galaxy in galaxies_outshined:
    nbins = galaxy.get_number_of_bins()
    new_nbins = len(np.unique(galaxy.pixedfit_nomin_map)) - 1
    plt.scatter(nbins, new_nbins)
    tot_bins += new_nbins
    '''
    figg, axx = plt.subplots(2, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
    # Plot old and new bin maps side by side - make colormap using nipy_spectral_r with different colors for each integer bin
    
    cmap = plt.cm.nipy_spectral_r
    cmap.set_under('white')
    cmap.set_bad('white')
    # Plot old bin map
    axx[0].imshow(galaxy.pixedfit_map, cmap=cmap, vmin=0, vmax=nbins)
    axx[0].set_title('Old Bin Map')
    # Plot new bin map
    axx[1].imshow(galaxy.pixedfit_nomin_map, cmap=cmap, vmin=0, vmax=new_nbins)
    axx[1].set_title('New Bin Map')

    plt.show()
    '''
    #galaxy.pixedfit_plot_binmap()

plt.plot([0, 40], [0, 40], color='black', linestyle='--', linewidth=1)

ax.set_xlabel('Old Bin Number')
ax.set_ylabel('New Bin Number')
print(f'Old total_bins {galaxies_outshined.total_number_of_bins()}')
print(f'Total bins: {tot_bins}')




In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
masses = []
bins = []
for galaxy in galaxies_outshined:
    old_mass = galaxy.get_total_resolved_property('CNST_SFH_RESOLVED_P', 'stellar_mass')
    new_mass = galaxy.get_total_resolved_property('CNST_SFH_RESOLVED_NOMIN', 'stellar_mass')

    delta_mass = new_mass[1] - old_mass[1]
    masses.append(delta_mass)

    nbins_old = galaxy.get_number_of_bins()
    nbins_new = galaxy.get_number_of_bins('pixedfit_nomin')
    bins.append(nbins_new-nbins_old)


ax.scatter(bins, masses)
ax.set_xscale('log')



In [ ]:
galaxies_outshined.comparison_plot("CNST_SFH_RESOLVED_P", "CNST_SFH_RESOLVED_VORONOI", aperture_label='RESOLVED', color_by='redshift')
galaxies_outshined.comparison_plot("CNST_SFH_RESOLVED_P", "CNST_SFH_RESOLVED_NOMIN", aperture_label='RESOLVED', color_by='redshift')
galaxies_outshined.comparison_plot("CNST_SFH_RESOLVED_NOMIN", "BURSTY_SFH_RESOLVED_NOMIN", aperture_label='RESOLVED', color_by='redshift')

In [ ]:
galaxies_outshined.comparison_plot("photoz_delayed", "CNST_SFH_RESOLVED_NOMIN", color_by='redshift')
galaxies_outshined.comparison_plot("photoz_delayed", "BURSTY_SFH_RESOLVED_NOMIN", color_by='redshift')
galaxies_outshined.comparison_plot("photoz_delayed", "CNST_SFH_RESOLVED_P", color_by='redshift')
galaxies_outshined.comparison_plot("photoz_delayed", "CNST_SFH_RESOLVED_VORONOI", color_by='redshift')
galaxies_outshined.comparison_plot("photoz_delayed", "BURSTY_SFH_RESOLVED_VORONOI", color_by='redshift')



In [ ]:
[galaxy.load_bagpipes_results('CNST_SFH_RESOLVED_VORONOI', run_dir='../pipes/') for galaxy in galaxies_not_outshined]

In [ ]:

mask = []
limit = 8.5
for galaxy in galaxies:
    try:
        new_mass = galaxy.get_total_resolved_property('CNST_SFH_RESOLVED_VORONOI', 'stellar_mass')[1]
        mask.append(True if new_mass < limit else False)
    except:
        mask.append(False)

selected_galaxies = galaxies[mask]

regions = ['pixedfit', 'pixedfit_nomin', 'voronoi']
resolved_run = 'CNST_SFH_RESOLVED'
bagpipes_runs = ["photoz_cnst", "photoz_lognorm", "photoz_delayed", "photoz_continuity", "photoz_continuity_bursty"]
resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']



print(' & '.join(regions), r'\\')
for bagpipes_run in bagpipes_runs:
    delta_masses_run = []
    for resolved_run in resolved_runs:
        delta_masses = []
        for galaxy in selected_galaxies:
            try:
                new_mass = galaxy.get_total_resolved_property(resolved_run, 'stellar_mass')
            except:
                continue
            integrated_mass = galaxy.sed_fitting_table['bagpipes'][bagpipes_run]['stellar_mass_50'][0]
            delta_mass = new_mass[1] - integrated_mass
            delta_masses.append(delta_mass)
        delta_masses = np.array(delta_masses)
        #print(region, resolved_run, bagpipes_run)
        delta_masses_run.append(f'{np.log10(np.nanmedian(10**delta_masses)):.2f} ({np.log10(np.nanmean(10**delta_masses)):.2f})')
    print(bagpipes_run, ' & '.join(delta_masses_run), r'\\')


